In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import glob as gb
from PIL import Image

os.sys.path.append('../')

#from facenet_pytorch import MTCNN, extract_face
#from utils .align import AlignDlib

#import dlib
# To compile dlib
# sudo apt install -y cmake
# pip install dlib

In [42]:
# Creating auxiliary folders
root_data = '/media/angelo/DATEN/Datasets/QuisCampi-ICBRW/ICB-RW/'

if not os.path.exists(root_data + 'gallery/'):
    os.mkdir(root_data + 'gallery/')

if not os.path.exists(root_data + 'probe/'):
    os.mkdir(root_data + 'probe/')
    
if not os.path.exists(root_data + 'gallery-margin/'):
    os.mkdir(root_data + 'gallery-margin/')

if not os.path.exists(root_data + 'probe-margin/'):
    os.mkdir(root_data + 'probe-margin/')

In [43]:
# Saving gallery faces in gallery folder

with open(root_data + 'annotations_GalleryImages.txt') as f:
    for line in f:
        subject, x, y, width, height = line.split()
        if 'f' in subject.split('.')[0]: # Only segment frontal images since we're working only with them
            img = Image.open(root_data + 'icbrw_GalleryImages/' + subject)
            x, y, width, height = int(x), int(y), int(width), int(height)
            img = img.crop((x, y, x+width, y+height))
            #img.show()
            #break
            img.save(root_data + 'gallery/' + subject)

In [44]:
# Saving probe faces in probe folder

with open(root_data + 'annotations_ProbeImages.txt') as f:
    for line in f:
        subject, x, y, width, height = line.split()
        img = Image.open(root_data + 'icbrw_ProbeImages/' + subject)
        x, y, width, height = int(x), int(y), int(width), int(height)
        img = img.crop((x, y, x+width, y+height))
        #img.show()
        #break
        img.save(root_data + 'probe/' + subject)

In [45]:
# Saving gallery faces with 30% margin in gallery folder

with open(root_data + 'annotations_GalleryImages.txt') as f:
    for line in f:
        subject, x, y, width, height = line.split()
        if 'f' in subject.split('.')[0]: # Only segment frontal images since we're working only with them
            img = Image.open(root_data + 'icbrw_GalleryImages/' + subject)
            x, y, width, height = int(x), int(y), int(width), int(height)
            img = img.crop((0.85*x, 0.85*y, 1.15*(x+width), 1.15*(y+height)))
            #img.show()
            #break
            img.save(root_data + 'gallery-margin/' + subject)

In [46]:
# Saving probe faces with 30% margin in probe folder

with open(root_data + 'annotations_ProbeImages.txt') as f:
    for line in f:
        subject, x, y, width, height = line.split()
        img = Image.open(root_data + 'icbrw_ProbeImages/' + subject)
        x, y, width, height = int(x), int(y), int(width), int(height)
        img = img.crop((0.85*x, 0.85*y, 1.15*(x+width), 1.15*(y+height)))
        #img.show()
        #break
        img.save(root_data + 'probe-margin/' + subject)

In [ ]:
# In case annotation files are not available
# MTCNN detection
mtcnn = MTCNN(keep_all=True)

for i, subject in enumerate(list_of_people):
    
    face_id = '00' + subject + '_'
    gallery_faces_path = root_data + 'gallery/'
    probe_faces_path = root_data + 'probe/'
    probe_counter = 1
    
    if not os.path.exists(gallery_faces_path):
        os.mkdir(gallery_faces_path) 
    
    if not os.path.exists(probe_faces_path):
        os.mkdir(probe_faces_path) 
    
    for image in os.listdir(probe_faces_path + subject):
        
        image_name = os.path.splitext(image)[0]
        image_path = probe_faces_path + subject + '/' + image
        image_RGB = Image.open(image_path)
        
        faces, _ = mtcnn.detect(image_RGB)
        isEmpty = (str(type(faces)) == "<class 'NoneType'>")

        if isEmpty: # Checking if faces were detected
            print('Subject ' + face_id + ' not complete.')

        if not isEmpty: 
            if image_name in ['f','F']:
                extract_face(image_RGB, faces[0], image_resize=False,
                             save_path=gallery_faces_path + face_id + str(image_name).lower() + '.jpg')
            else:
                extract_face(image_RGB, faces[0], image_resize=False,
                             save_path=probe_faces_path + subject + '/' + face_id + str(probe_counter) + '.jpg')
                probe_counter+=1

    print('Processed subject {}/{}'.format(i+1,len(list_of_people)))

In [ ]:
# In case annotation files are not available
# DLIB detection

detector = dlib.get_frontal_face_detector()

for i, subject in enumerate(list_of_people):
    
    face_id = '00' + subject + '_'
    gallery_faces_path = root_data + 'gallery/'
    probe_faces_path = root_data + 'probe/'
    probe_counter = 0 # For naming according to ICB_RW official data 
    
    if not os.path.exists(gallery_faces_path):
        os.mkdir(gallery_faces_path) 
    
    if not os.path.exists(probe_faces_path):
        os.mkdir(probe_faces_path) 
    
    for image in os.listdir(root_data + subject):
        
        image_name = os.path.splitext(image)[0]
        image_RGB = dlib.load_rgb_image(root_data + subject + '/' + image)
        faces = detector(image_RGB, 1)
        for face in faces: # We have only one face per image
            if image_name in ['f','r','l']:
                cv2.imwrite(gallery_faces_path + face_id + image_name + '.jpg', 
                            image_RGB[face.top():face.bottom(), 
                                   face.left():face.right()][...,::-1])
            else:
                probe_counter += 1
                cv2.imwrite(probe_faces_path + face_id + str(probe_counter) + '.jpg', 
                            image_RGB[face.top():face.bottom(), 
                                   face.left():face.right()][...,::-1])
    break